<a href="https://colab.research.google.com/github/AnsonQiu/Box-Model/blob/main/Assignment_0_aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd
import numpy as np






In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [3]:
#Describe your answer below.
#Answer 1: The first command retrieves data from the file and separate each column
#          of the dataset by the character "|". The second command selects dataset with
#          the rows with the column "Test Issue" entry == 'N'. The third command creates
#          a set of values for the new dataset's values in the column 'NASDAQ Symbol'. The
#          next three commands basically does the samething except changing the dataset. 

In [4]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2023-01-27 18:13:26--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2023-01-27 18:13:26--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc31f891d2e3b1f1c1c119970235.dl.dropboxusercontent.com/cd/0/get/B1WyxUTxN9cz6eZOBJ3zndQqyBRH8PKVGLIAXn6msBPLPCl8FKlqvvSGng-7zKbuL5FFJKzX6jsIm9PxqvEXvWhogrg-EUN3g5EiZ_7qHFTMVz-1NJzOqlpKAC_ufreLZVhg6vdr8RQ9zokRXk5Bu7tSNp5_pJU5MTWnccTbYud9ZQ/file?dl=1# [following]
--2023-01-27 18:13:26--  https://uc31f891d2e3b1f1c1c119970235.dl.dropboxusercontent.com/cd/0/get/B1WyxUTxN9cz6eZOBJ3zndQqyBRH8PKVGLIAXn6msBPLPCl8FKlqvvSGng-7zKbuL5FFJKzX

In [5]:
!ls stocks.parquet -l #downloaded file

-rw-rw-r-- 1 ubuntu ubuntu 678640589 Jan 27 18:13 stocks.parquet


In [6]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list. 

In [7]:
data=pd.read_parquet('stocks.parquet')

In [8]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock



,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [9]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

In [10]:
dataS = data[["date", "adj_close", "symbol"]]
dataS['date'] = pd.to_datetime(dataS['date'], format='%Y-%m-%d')
dataS.sort_values(by='date', inplace= True)
df = dataS.groupby('symbol')['date', 'adj_close'].agg(['first', 'last'])
dfX = df['adj_close', 'first']
dfY = df['adj_close', 'last']
dfZ = df['date', 'last'] - df['date', 'first']

/tmp/ipykernel_2219/718787043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataS['date'] = pd.to_datetime(dataS['date'], format='%Y-%m-%d')
/tmp/ipykernel_2219/718787043.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataS.sort_values(by='date', inplace= True)
/tmp/ipykernel_2219/718787043.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = dataS.groupby('symbol')['date', 'adj_close'].agg(['first', 'last'])


In [11]:
Z = dfZ.dt.days
Z.head(5)
dfR = dfY/dfX/Z
dfR.replace([np.inf, -np.inf], np.nan, inplace=True)
dfR.dropna(inplace=True)
dfR.sort_values(ascending=False, inplace=True)
print(dfR.head(20))
# The top 20 stocks have been printed

symbol
NVO      3.302876e+06
NKE      2.959418e+03
DCI      3.869564e+02
TJX      2.063167e+02
WTRG     7.221605e+00
TTC      6.224708e+00
USB      4.078555e+00
MCD      3.374097e+00
PII      2.554223e+00
HD       1.506773e+00
IQMD     1.001014e+00
AOGOU    9.990020e-01
PCCT     9.959839e-01
ETN      9.076372e-01
FRLAW    8.888889e-01
HAS      7.035260e-01
KO       6.680578e-01
MO       6.485381e-01
KOL      5.786728e-01
HRL      5.406652e-01
dtype: float64


In [12]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS